# VAE-LSTM [PyTorch]

This notebook is based on the codes below

https://github.com/Khamies/LSTM-Variational-AutoEncoder/tree/main

https://github.com/Khamies/LSTM-Variational-AutoEncoder/blob/main/model.py

We use the trained model to perform inferences (no training is done).

In [1]:
import torch

In [2]:
torch.__version__

'2.2.2'

In [3]:
class LSTM_VAE(torch.nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, latent_size, num_layers=1):
        super(LSTM_VAE, self).__init__()
    
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        # Variables
        self.num_layers = num_layers
        self.lstm_factor = num_layers
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.dictionary = PTB(data_dir="./data", split="train", 
                              create_data= False, max_sequence_length= 60)

        # X: bsz * seq_len * vocab_size 
        # Embedding
        self.embed = torch.nn.Embedding(num_embeddings= self.vocab_size,
                                        embedding_dim= self.embed_size)

        # Encoder Part: define the layers
        self.encoder_lstm = torch.nn.LSTM(input_size= self.embed_size,
                                          hidden_size= self.hidden_size,
                                          batch_first=True, num_layers= self.num_layers)
        self.mean = torch.nn.Linear(in_features= self.hidden_size * self.lstm_factor,
                                    out_features= self.latent_size)
        self.log_variance = torch.nn.Linear(in_features= self.hidden_size * self.lstm_factor, 
                                            out_features= self.latent_size)

        # Decoder Part: define the layers                                       
        self.init_hidden_decoder = torch.nn.Linear(in_features= self.latent_size,
                                                   out_features= self.hidden_size * self.lstm_factor)
        self.decoder_lstm = torch.nn.LSTM(input_size= self.embed_size, hidden_size= self.hidden_size, 
                                          batch_first = True, num_layers = self.num_layers)
        self.output = torch.nn.Linear(in_features= self.hidden_size * self.lstm_factor, 
                                      out_features= self.vocab_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)
    
    #initialize the hidden cells [h,c] of lstm
    def init_hidden(self, batch_size):
        hidden_cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
        state_cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
        return (hidden_cell, state_cell)
    
    #create the Embedding layer
    def get_embedding(self, x):
        x_embed = self.embed(x)
        # Total length for pad_packed_sequence method = maximum sequence length
        maximum_sequence_length = x_embed.size(1)
        return x_embed, maximum_sequence_length
    
    def encoder(self, packed_x_embed,total_padding_length, hidden_encoder):
        # pad the packed input.
        packed_output_encoder, hidden_encoder = self.encoder_lstm(packed_x_embed, hidden_encoder)
        output_encoder, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_output_encoder, 
                                                                   batch_first=True, 
                                                                   total_length= total_padding_length)

        # Extimate the mean and the variance of q(z|x)
        mean = self.mean(hidden_encoder[0])
        log_var = self.log_variance(hidden_encoder[0])
        std = torch.exp(0.5 * log_var)   # e^(0.5 log_var) = var^0.5

        # Generate a unit gaussian noise.
        batch_size = output_encoder.size(0)
        seq_len = output_encoder.size(1)
        noise = torch.randn(batch_size, self.latent_size).to(self.device)

        z = noise * std + mean

        return z, mean, log_var, hidden_encoder
    
    def decoder(self, z, packed_x_embed, total_padding_length=None):
        hidden_decoder = self.init_hidden_decoder(z)
        hidden_decoder = (hidden_decoder, hidden_decoder)

        # pad the packed input.
        packed_output_decoder, hidden_decoder = self.decoder_lstm(packed_x_embed,hidden_decoder) 
        output_decoder, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_output_decoder,
                                                                   batch_first=True, 
                                                                   total_length= total_padding_length)

        x_hat = self.output(output_decoder)
        x_hat = self.log_softmax(x_hat)
        return x_hat

    def forward(self, x, sentences_length, hidden_encoder):  
        """
            x : bsz * seq_len
          hidden_encoder: ( num_lstm_layers * bsz * hidden_size, num_lstm_layers * bsz * hidden_size)
        """
        # Get Embeddings
        x_embed, maximum_padding_length = self.get_embedding(x)
        # Packing the input
        packed_x_embed = torch.nn.utils.rnn.pack_padded_sequence(input= x_embed, 
                                                                 lengths= sentences_length, 
                                                                 batch_first=True, enforce_sorted=False)
        # Encoder
        z, mean, log_var, hidden_encoder = self.encoder(packed_x_embed, maximum_padding_length, hidden_encoder)
        # Decoder
        x_hat = self.decoder(z, packed_x_embed, maximum_padding_length)
        return x_hat, mean, log_var, z, hidden_encoder
    
    def inference(self, n_samples, sos, z):
        # generate random z 
        batch_size = 1
        seq_len = 1
        idx_sample = []

        input = torch.Tensor(1, 1).fill_(self.dictionary.get_w2i()[sos]).long().to(self.device)

        hidden = self.init_hidden_decoder(z)
        hidden = (hidden, hidden)

        for i in range(n_samples):
          input = self.embed(input)
          output,hidden = self.decoder_lstm(input, hidden)
          output = self.output(output)
          output = self.log_softmax(output)
          output = output.exp()
          _, s = torch.topk(output, 1)
          idx_sample.append(s.item())
          input = s.squeeze(0)

        w_sample = [self.dictionary.get_i2w()[str(idx)] for idx in idx_sample]
        w_sample = " ".join(w_sample)

        return w_sample


# Define VAE Loss 

In [4]:
class VAE_Loss(torch.nn.Module):

  def __init__(self):
    super(VAE_Loss, self).__init__()
    self.nlloss = torch.nn.NLLLoss()
  
  def KL_loss (self, mu, log_var, z):
    kl = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    kl = kl.sum(-1)  # to go from multi-dimensional z 
    #to single dimensional z : (batch_size x latent_size) ---> (batch_size) 
                            # i.e Z = [ [z1_1, z1_2 , ...., z1_lt] ] ------> z = [ z1] 
                            #         [ [z2_1, z2_2, ....., z2_lt] ]             [ z2]
                            #                   .                                [ . ]
                            #                   .                                [ . ]
                            #         [[zn_1, zn_2, ....., zn_lt] ]              [ zn]
                                                                      
                            #        lt=latent_size 
    kl = kl.mean()                                                                    
    return kl

  def reconstruction_loss(self, x_hat_param, x):
    x = x.view(-1).contiguous()
    x_hat_param = x_hat_param.view(-1, x_hat_param.size(2))
    recon = self.nlloss(x_hat_param, x)
    return recon
  

  def forward(self, mu, log_var,z, x_hat_param, x):
    kl_loss = self.KL_loss(mu, log_var, z)
    recon_loss = self.reconstruction_loss(x_hat_param, x)
    elbo = kl_loss + recon_loss # we use + because recon loss is a NLLoss (cross entropy) and it's negative in its own, and in the ELBO equation we have
                              # elbo = KL_loss - recon_loss, therefore, ELBO = KL_loss - (NLLoss) = KL_loss + NLLoss
    return elbo, kl_loss, recon_loss

# Define train loop

https://github.com/Khamies/LSTM-Variational-AutoEncoder/blob/main/train.py

In [5]:
class Trainer:

    def __init__(self, train_loader, test_loader, model, loss, optimizer) -> None:
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.model = model
        self.loss = loss
        self.optimizer = optimizer
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.interval = 200


    def train(self, train_losses, epoch, batch_size, clip) -> list:  
        # Initialization of RNN hidden, and cell states.
        states = self.model.init_hidden(batch_size) 

        for batch_num, batch in enumerate(self.train_loader): # loop over the data, and jump with step = bptt.
            # get the labels
            source, target, source_lengths = get_batch(batch)
            source = source.to(self.device)
            target = target.to(self.device)
            x_hat_param, mu, log_var, z, states = self.model(source,source_lengths, states)

            # detach hidden states
            states = states[0].detach(), states[1].detach()

            # compute the loss
            mloss, KL_loss, recon_loss = self.loss(mu = mu, log_var = log_var, z = z,
                                                   x_hat_param = x_hat_param , x = target)
            train_losses.append((mloss , KL_loss.item(), recon_loss.item()))
            mloss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), clip)
            self.optimizer.step()
            self.optimizer.zero_grad()
            if batch_num % self.interval == 0 and batch_num > 0:
  
                print('| epoch {:3d} | elbo_loss {:5.6f} | kl_loss {:5.6f} | recons_loss {:5.6f} '.format(
                    epoch, mloss.item(), KL_loss.item(), recon_loss.item()))

        return train_losses

    def test(self, test_losses, epoch, batch_size) -> list:
        with torch.no_grad():
            states = self.model.init_hidden(batch_size) 

            for batch_num, batch in enumerate(self.test_loader): # loop over the data, and jump with step = bptt.
                # get the labels
                source, target, source_lengths = get_batch(batch)
                source = source.to(self.device)
                target = target.to(self.device)

                x_hat_param, mu, log_var, z, states = self.model(source,source_lengths, states)

                # detach hidden states
                states = states[0].detach(), states[1].detach()

                # compute the loss
                mloss, KL_loss, recon_loss = self.loss(mu = mu, log_var = log_var,
                                                       z = z, x_hat_param = x_hat_param , x = target)

                test_losses.append((mloss , KL_loss.item(), recon_loss.item()))

                # Statistics.
                # if batch_num % 20 ==0:
                #   print('| epoch {:3d} | elbo_loss {:5.6f} | kl_loss {:5.6f} | recons_loss {:5.6f} '.format(
                #         epoch, mloss.item(), KL_loss.item(), recon_loss.item()))

            return test_losses

# Utilities

In [6]:
import matplotlib.pyplot as plt
from data.ptb import PTB
from settings import training_setting
import torch

def get_batch(batch):
  sentences = batch["input"]
  target = batch["target"]
  sentences_length = batch["length"]
  return sentences, target, sentences_length

def plot_elbo(losses, mode):
    elbo_loss = list(map(lambda x: x[0], losses))
    kl_loss = list(map(lambda x: x[1], losses))
    recon_loss = list(map(lambda x: x[2], losses))

    losses = {"elbo": elbo_loss, "kl": kl_loss, "recon": recon_loss}
    print(losses)
    for key in losses.keys():
        plt.plot(losses.get(key), label=key+"_" + mode)
    plt.legend()
    plt.show()


def get_latent_codes(dataloader, model, batch_size):
  hidden = model.init_hidden(batch_size)
  Z = []
  with torch.no_grad():
    for batch in dataloader:
        x, t, leng = batch.get("input"), batch.get(
            "target"), batch.get("length")
        x = x.to(model.device)
        t.to(model.device)
        _, _, _, z, _ = model(x, leng, hidden)
        Z.append(z)
    Z = torch.cat(Z[:-1])
    Z = Z.reshape(-1, Z.size(2))
    return Z

def visualize_latent_codes(z):
    z = z.squeeze(0).t().contiguous()
    n_z = z.size(0)
    n = n_z//2
    fig = plt.figure(figsize=(20, 6), dpi=80)
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    for i in range(1, 2*n):
        ax = fig.add_subplot(2, n, i)
        ax.hist(z[i].tolist())
    plt.show()

def interpolate(model, n_interpolations, sos, sequence_length):
  z1 = torch.randn((1,1,model.latent_size)).to(model.device)
  z2 = torch.randn((1,1,model.latent_size)).to(model.device)
  text1 = model.inference(sequence_length , sos, z1)
  text2 = model.inference(sequence_length , sos, z2)
  alpha_s = torch.linspace(0,1,n_interpolations)
  interpolations = torch.stack([alpha*z1 + (1-alpha)*z2  for alpha in alpha_s])
  samples = [model.inference(sequence_length , sos, z) for z in interpolations]
  return samples, text1, text2

# Load data 

https://github.com/Khamies/LSTM-Variational-AutoEncoder/blob/main/main.py

In [7]:
#parser.add_argument("--batch_size", type=str, default="32")
#parser.add_argument("--bptt", type=str,default="60")
#parser.add_argument("--embed_size", type=str, default="300") 
#parser.add_argument("--hidden_size", type=str, default="256")
#parser.add_argument("--latent_size", type=str, default="16")
#parser.add_argument("--lr", type=str, default="0.001")
bptt = 60
batch_size = 32
embed_size = 300
hidden_size = 256 #(LSTM hidden size)
latent_size = 16 #(VAE latent size)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
# Load the data
train_data = PTB(data_dir="./data", split="train", create_data= False, max_sequence_length= bptt)
test_data = PTB(data_dir="./data", split="test", create_data= False, max_sequence_length=bptt)
valid_data = PTB(data_dir="./data", split="valid", create_data= False, max_sequence_length= bptt)

# Batchify the data
train_loader = torch.utils.data.DataLoader( dataset= train_data, batch_size=batch_size, shuffle= True)
test_loader = torch.utils.data.DataLoader( dataset= test_data, batch_size= batch_size, shuffle= True)
valid_loader = torch.utils.data.DataLoader( dataset= valid_data, batch_size= batch_size, shuffle= True)

In [9]:
vocab_size = train_data.vocab_size
vocab_size

9839

# Check data stored in loaders

In [10]:
len(train_loader), len(test_loader), len(valid_loader)

(1315, 118, 106)

In [11]:
list(enumerate(train_loader))[:1]

[(0,
  {'input': tensor([[   2,   87, 7345,  ...,    0,    0,    0],
           [   2,   35,   19,  ...,    0,    0,    0],
           [   2,   10, 3536,  ...,    0,    0,    0],
           ...,
           [   2,  438, 3199,  ...,    0,    0,    0],
           [   2,  224,  751,  ...,    0,    0,    0],
           [   2,   87,  664,  ...,    0,    0,    0]]),
   'target': tensor([[  87, 7345,   13,  ...,    0,    0,    0],
           [  35,   19,   10,  ...,    0,    0,    0],
           [  10, 3536,   79,  ...,    0,    0,    0],
           ...,
           [ 438, 3199,   77,  ...,    0,    0,    0],
           [ 224,  751,  817,  ...,    0,    0,    0],
           [  87,  664,    5,  ...,    0,    0,    0]]),
   'length': tensor([36, 24,  7, 21, 19, 28, 29, 24, 10,  4, 33, 37,  8, 24, 44, 30, 45, 30,
           14, 26, 13, 31, 16, 33, 31, 22, 54, 31, 16, 24, 14, 45])})]

In [12]:
len(list(enumerate(train_loader))[0][1]['input']),list(enumerate(train_loader))[0][1]['input']

(32,
 tensor([[   2,  208, 1935,  ...,    0,    0,    0],
         [   2,  160, 1759,  ...,    0,    0,    0],
         [   2,   10, 1171,  ...,    0,    0,    0],
         ...,
         [   2, 1485,   13,  ...,    0,    0,    0],
         [   2,   57,   98,  ...,    0,    0,    0],
         [   2,   10,  409,  ...,    0,    0,    0]]))

In [13]:
for i, seq in list(enumerate(train_loader)):
    print(i)
    print(seq['input'])
    print(seq['target'])
    break

0
tensor([[   2,   87, 9320,  ...,    0,    0,    0],
        [   2,   10,   36,  ...,    0,    0,    0],
        [   2, 5714,    1,  ...,    0,    0,    0],
        ...,
        [   2,  533, 1161,  ...,    0,    0,    0],
        [   2,    1,  160,  ...,    0,    0,    0],
        [   2,   13,  810,  ...,    0,    0,    0]])
tensor([[  87, 9320,    5,  ...,    0,    0,    0],
        [  10,   36, 3839,  ...,    0,    0,    0],
        [5714,    1,  208,  ...,    0,    0,    0],
        ...,
        [ 533, 1161, 4952,  ...,    0,    0,    0],
        [   1,  160,  385,  ...,    0,    0,    0],
        [  13,  810,  977,  ...,    0,    0,    0]])


In [14]:
# Extract a single sequence for passing through to the model 
X_0 = list(enumerate(train_loader))[0][1]['input']
X0_sentences_length= list(enumerate(train_loader))[0][1]['length']
X_0, X0_sentences_length

(tensor([[   2,   87,   88,  ...,    0,    0,    0],
         [   2, 3729,   10,  ...,    0,    0,    0],
         [   2, 6172, 3013,  ...,    0,    0,    0],
         ...,
         [   2, 1492, 2463,  ...,    0,    0,    0],
         [   2, 1696,  139,  ...,    0,    0,    0],
         [   2, 5914,   77,  ...,    0,    0,    0]]),
 tensor([ 9, 14, 34, 11, 34, 19, 10, 25, 11, 20, 19, 11, 20, 31, 12, 24, 15,  7,
         13, 32, 17, 27, 16, 16, 17, 21, 20, 17, 31,  7, 15, 21]))

# Shape tracing

In [15]:
model = LSTM_VAE(vocab_size = vocab_size, embed_size = embed_size,
                 hidden_size = hidden_size, latent_size = latent_size).to(device)
model

LSTM_VAE(
  (embed): Embedding(9839, 300)
  (encoder_lstm): LSTM(300, 256, batch_first=True)
  (mean): Linear(in_features=256, out_features=16, bias=True)
  (log_variance): Linear(in_features=256, out_features=16, bias=True)
  (init_hidden_decoder): Linear(in_features=16, out_features=256, bias=True)
  (decoder_lstm): LSTM(300, 256, batch_first=True)
  (output): Linear(in_features=256, out_features=9839, bias=True)
  (log_softmax): LogSoftmax(dim=2)
)

In [16]:
print(f'Initial sequence in a batch: {X_0.shape}')
hidden_encoder = model.init_hidden(batch_size) 
print(f'Hidden encoder states: {hidden_encoder[0].shape}')
x_embed, max_pad_len = model.get_embedding(X_0)
print(f'After embedding: {x_embed.shape}')
print(f'Max seq length: {max_pad_len}')

# Packing the input
packed_x_embed = torch.nn.utils.rnn.pack_padded_sequence(input= x_embed, 
                                                        lengths= X0_sentences_length, 
                                                        batch_first=True, 
                                                        enforce_sorted=False)
#print(f'after packing {packed_x_embed}')
# Encoder
z, mean, log_var, hidden_encoder = model.encoder(packed_x_embed, 
                                        max_pad_len, 
                                        hidden_encoder)
print(f'z shape: {z.shape}')
print(f'mu shape: {mean.shape}')
print(f'log var shape: {log_var.shape}')
# Decoder
x_hat = model.decoder(z, packed_x_embed, max_pad_len)
print(f'x_hat shape: {x_hat.shape}')

Initial sequence in a batch: torch.Size([32, 60])
Hidden encoder states: torch.Size([1, 32, 256])
After embedding: torch.Size([32, 60, 300])
Max seq length: 60
z shape: torch.Size([1, 32, 16])
mu shape: torch.Size([1, 32, 16])
log var shape: torch.Size([1, 32, 16])
x_hat shape: torch.Size([32, 60, 9839])


In [17]:
x_hat[0][0]

tensor([-9.2340, -9.1213, -9.3122,  ..., -9.3198, -9.1470, -9.2381],
       grad_fn=<SelectBackward0>)

# Training

In [18]:
#Skip training
#Loss = VAE_Loss()
#optimizer = torch.optim.Adam(model.parameters(), lr= training_setting["lr"])
#trainer = Trainer(train_loader, test_loader, model, Loss, optimizer)

#train_losses = []
#test_losses = []
#for epoch in range(training_setting["epochs"]):
#    print("Epoch: ", epoch)
#    print("Training.......")
#    train_losses = trainer.train(train_losses, epoch, training_setting["batch_size"], training_setting["clip"])
#    print("Testing.......")
#    test_losses = trainer.test(test_losses, epoch, training_setting["batch_size"])

#plot_elbo(train_losses, "train")
#plot_elbo(test_losses, "test")

# Load trained model

In [19]:
checkpoint = torch.load("models/LSTM_VAE.pt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [20]:
# Run the trained model on a single batch
# Encoder
z, mean, log_var, hidden_encoder = model.encoder(packed_x_embed, 
                                        max_pad_len, 
                                        hidden_encoder)
print(f'z shape: {z.shape}')
print(f'mu shape: {mean.shape}')
print(f'log var shape: {log_var.shape}')

z shape: torch.Size([1, 32, 16])
mu shape: torch.Size([1, 32, 16])
log var shape: torch.Size([1, 32, 16])


In [21]:
z

tensor([[[ 0.1077,  1.1136, -0.5670, -0.6029, -1.0720,  1.3184,  0.6331,
          -0.1960,  0.6857,  0.8333,  0.8115,  1.0224, -0.8659, -1.1526,
           0.5827,  0.3481],
         [-1.2044,  1.2788, -2.3048, -0.8692, -0.8205,  0.2371, -1.8795,
           0.7737,  0.9681, -1.2852, -0.8859,  0.4227, -1.3879, -1.8224,
           0.3157,  0.1263],
         [ 1.0516,  0.2249,  0.0330,  0.7451, -1.1266, -2.7080,  0.5207,
          -1.3420,  0.2020, -0.1656, -0.5253,  0.4150,  0.3043,  1.1146,
           0.8488, -0.3908],
         [-1.1080,  0.0394,  1.6549, -2.1516,  0.4683,  0.1034,  0.3168,
          -1.8934, -0.2395,  0.1188,  0.0279, -0.7378,  0.7542, -0.6265,
          -0.0839, -0.4535],
         [ 0.1416,  0.4957, -1.5157, -0.7472,  1.3484,  0.4683, -0.6200,
          -0.2064,  0.0377, -0.8698, -0.7952,  0.1925,  0.1340,  0.9371,
          -0.1409,  0.5852],
         [-0.5278,  0.7211, -0.9456,  1.7364,  0.1067,  2.0498,  1.8641,
          -0.1112, -0.6168, -0.3236,  1.0144, -0.380

In [22]:
# Decoder
x_hat = model.decoder(z, packed_x_embed, max_pad_len)
print(f'x_hat shape: {x_hat.shape}')

x_hat shape: torch.Size([32, 60, 9839])


In [23]:
x_hat[0][0]

tensor([ -8.8279,  -5.9137, -19.1463,  ..., -15.9236, -16.8142, -11.2007],
       grad_fn=<SelectBackward0>)

In [24]:
z1 = torch.randn(1,1,latent_size).to(device)
z2 = torch.randn(1,1,latent_size).to(device)
print(z1.shape)
sos = "<sos>"
unk = "<unk>"
sample1 = model.inference(25 , sos, z1)
sample2 = model.inference(25 , sos, z2)

print(sample1)
print(sample2)

torch.Size([1, 1, 16])
the company said it will close the <unk> offering for its $ n million acquisition of first union corp . which has relied heavily on
the <unk> <unk> <unk> is a <unk> and <unk> <unk> <eos> who <unk> them <eos> <eos> <eos> and they are on their business <eos> <eos>


In [25]:
def interpolate(model, n_interpolations, sos, sequence_length):
  # # Get input.
  text1 = model.inference(sequence_length , sos, z1)
  text2 = model.inference(sequence_length , sos, z2)
  alpha_s = torch.linspace(0,1,n_interpolations)
  interpolations = torch.stack([alpha*z1 + (1-alpha)*z2  for alpha in alpha_s])
  samples = [model.inference(sequence_length , sos, z) for z in interpolations]
  return samples, text1, text2

In [26]:
samples, text1, text2 = interpolate(model, 20, "company",  10)
print("First sentence:", text1)
print("Second sentence:", text2)

for sample in samples: print(sample)

First sentence: officials holders enfield chief of the american association of operations
Second sentence: <pad> a leading employee presidents and the carrier has moved
<pad> a leading employee presidents and the carrier has moved
<pad> a leading employee presidents and the carrier has moved
<pad> a leading employee presidents and the carrier has moved
<pad> a leading employee presidents and the carrier has moved
<pad> a leading employee presidents of the commission a spokesman
<pad> a <unk> assembly plant at <unk> <unk> who had
<pad> a <unk> assembly plant at <unk> <unk> who had
<pad> a <unk> spokesman said <eos> <unk> <unk> inc .
<pad> a <unk> spokesman said <eos> <unk> <unk> inc .
officials holders enfield chief of the chain quite quite <unk>
officials holders enfield chief of the chain quite quite <unk>
officials holders enfield chief of the chain quite quite <unk>
officials holders enfield chief of the american association of operations
officials holders enfield chief of the ameri